Import necessary libraries

In [ ]:
# Import necessary libraries
import requests
import pandas as pd
import json
from datetime import datetime, timezone, timedelta
import time

# For Generate Unique Transaction Id for accesing API
import uuid

# MQTT server
import paho.mqtt.subscribe as subscribe
import paho.mqtt.client as mqtt

Import API Key

Import API Key Instructions

1. Go to the [NSW API Fuel documentation](https://api.nsw.gov.au/Product/Index/22#v-pills-doc) to get the **API Key** and **Authorization Header**.  
2. Subscribe to the Fuel API by creating an NSW API account:  
   - Create a new app with your chosen **App Name** and **App Description**.  
   - Under **Add an API Product**, select **Fuel API**.  
   - Scroll to the bottom of the page and click **Add app**.  
   - You will receive your **API Key**, **API Secret**, and **Authorization Header**.  
3. In the home directory of your assignment folder, create a file named `config_secret.py`.  
   - _Note:_ Place `config_secret.py` in the same directory as `COMP5339AS02.ipynb`.  
4. Open `config_secret.py` and add your `API_KEY` and `AuthorizationHeader`.  
   - A sample file has been provided as `config_secret_sample.py`.  

In [ ]:
import config_secret

# grab your key from config_secret.py
API_KEY = config_secret.API_KEY
AuthorizationHeader = config_secret.AuthorizationHeader

1. Data Retrieval

In [ ]:
# 1. Data Retrieval

# Fuel API Document
# https://api.nsw.gov.au/Product/Index/22#v-pills-doc

class FuelPriceCheckAPI:
    def __init__(self, API_KEY, AuthorizationHeader):
        self.url_base = "https://api.onegov.nsw.gov.au" # Base Url
        self.API_KEY = API_KEY
        self.AuthorizationHeader = AuthorizationHeader

    
    def get_datetime_now():
        '''
        Get current datetime in the API required format.
        '''

        return datetime.now(timezone.utc).strftime("%d/%m/%Y %I:%M:%S %p")
    

    def get_datetime_x_days_ago(x: int):
        '''
        Get x days ago datetime since current datetime in the API required format.
        '''

        return (datetime.now(timezone.utc) - timedelta(days=x)).strftime("%d/%m/%Y %I:%M:%S %p")
    

    def get_unique_transactionId():
        '''
        Get Unique Transaction Id.
        '''

        return str(uuid.uuid4())


    def get_accesstoken(self):
        '''
        The client can request an access token using only its client credentials (or other supported means of authentication) when the client is requesting access to the protected resources under its control, or those of another resource owner that have been previously arranged with the authorization server.
        '''

        # Config url api prameters for getting accesstoken
        url_subpart = "/oauth/client_credential/accesstoken"

        url = self.url_base + url_subpart

        query_getAccessToken = {"grant_type":"client_credentials"}

        headers_getAccessToken = {'Authorization': self.AuthorizationHeader}

        time.sleep(5)

        # Get the web result for accesstoken
        response = requests.get(url, headers=headers_getAccessToken, params=query_getAccessToken)

        # Get the accesstoken from the web result
        access_token = response.json()["access_token"]

        return access_token


    def getFuelPriceReference(self, modified_since_datetime=None):
        '''
        modified_since_datetime: The datetime UTC date and time of request. Format dd/MM/yyyy hh:mm:ss AM/PM. Default 1 day before current time.
        
        Meaning:
        modified_since_datetime: The if-modified-since header tells the API to return the reference data only if it has changed since the given UTC timestamp, otherwise it returns “Not Modified” to save bandwidth.
         
        Description:
        Returns lists of reference data to be used in subsequent requests. It is recommended to execute this call in a separate api client as response can be over 2 mb. This API returns data for NSW.
        '''

        # Config url api prameters for getting fuel price
        url_subpart = "/FuelCheckRefData/v1/fuel/lovs"

        url = self.url_base + url_subpart

        # Config modified_since_datetime
        if modified_since_datetime is None:
            modified_since_datetime = FuelPriceCheckAPI.get_datetime_x_days_ago(1)

        headers = {
            'Authorization': f'Bearer {self.get_accesstoken()}',
            'Content-Type': 'application/json; charset=utf-8',
            'apikey': self.API_KEY,
            'transactionid': FuelPriceCheckAPI.get_unique_transactionId(),
            'requesttimestamp': FuelPriceCheckAPI.get_datetime_now(),
            'if-modified-since': modified_since_datetime
        }

        time.sleep(5)

        response = requests.get(url, headers=headers)

        return response


    def getFuelPrice(self):
        '''
        Returns all current fuel prices for all service stations. There may be restrictions on how often this API request can be made. It is recommended to execute this call in a separate api client as response can be over 2 mb. This API returns data for NSW.
        '''
        # Config url api prameters for getting fuel price
        url_subpart = "/FuelPriceCheck/v1/fuel/prices"

        url = self.url_base + url_subpart

        headers = {
            'Authorization': f'Bearer {self.get_accesstoken()}',
            'Content-Type': 'application/json; charset=utf-8',
            'apikey': self.API_KEY,
            'transactionid': FuelPriceCheckAPI.get_unique_transactionId(),
            'requesttimestamp': FuelPriceCheckAPI.get_datetime_now()
        }

        time.sleep(5)

        response = requests.get(url, headers=headers)

        return response


    def getNewFuelPrice(self):
        '''
        Returns all new current prices that have been submitted since the last "/fuelpricecheck/v1/fuel/prices" or "/fuelpricecheck/v1/fuel/prices/new" request using the apikey on the current day. This API returns data for NSW.
        '''
        # Config url api prameters for getting fuel price
        url_subpart = "/FuelPriceCheck/v1/fuel/prices/new"

        url = self.url_base + url_subpart

        headers = {
            'Authorization': f'Bearer {self.get_accesstoken()}',
            'Content-Type': 'application/json; charset=utf-8',
            'apikey': self.API_KEY,
            'transactionid': FuelPriceCheckAPI.get_unique_transactionId(),
            'requesttimestamp': FuelPriceCheckAPI.get_datetime_now()
        }

        time.sleep(5)

        response = requests.get(url, headers=headers)

        return response


In [ ]:
# Construct the class
fuelpriceAPI = FuelPriceCheckAPI(API_KEY, AuthorizationHeader)

In [ ]:
# To use the API, simply called fuelpriceAPI.getNewFuelPrice() or fuelpriceAPI.getFuelPrice() or fuelpriceAPI.getFuelPriceReference() depends on the usage
responseFromGetFuelPriceReference = fuelpriceAPI.getFuelPriceReference()

In [ ]:
print(json.dumps(responseFromGetFuelPriceReference.json(), indent=4, sort_keys=True))

In [ ]:
# To use the API, simply called fuelpriceAPI.getNewFuelPrice() or fuelpriceAPI.getFuelPrice() or fuelpriceAPI.getFuelPriceReference() depends on the usage
responseFromGetFuelPrice = fuelpriceAPI.getFuelPrice()

In [ ]:
# See the result
print(json.dumps(responseFromGetFuelPrice.json(), indent=4, sort_keys=True))

In [ ]:
# To use the API, simply called fuelpriceAPI.getNewFuelPrice() or fuelpriceAPI.getFuelPrice() or fuelpriceAPI.getFuelPriceReference() depends on the usage
responseFromGetNewFuelPrice = fuelpriceAPI.getNewFuelPrice()

In [ ]:
# See the result
print(json.dumps(responseFromGetNewFuelPrice.json(), indent=4, sort_keys=True))

API Calls
1. Create API class instance FuelPriceCheckAPI by entering argument API_KEY and AuthorizationHeader
2. Call API using the instance depends on usage
    - instance.getNewFuelPrice()
    - instance.getFuelPrice()
    - instance.getFuelPriceReference()

_Note:_ A FuelPriceCheckAPI instance has been created with the name fuelpriceAPI. simply called fuelpriceAPI.getNewFuelPrice() or fuelpriceAPI.getFuelPrice() or fuelpriceAPI.getFuelPriceReference() depends on the usage.

2. Data Integration and Storage

In [ ]:
# 2. Data Integration and Storage

3. Data Publishing via MQTT

In [ ]:
# 3. Data Publishing via MQTT

# Publisher


4. Data Subscribing and Visualisation

In [ ]:
# 4. Data Subscribing and Visualisation

5. Continuous Execution

In [ ]:
# 5. Continuous Execution